# Sentiment Analysis Ulasan Hotel Dengan Random Forest (Oversampling :SMOTE)

In [ ]:
# Install Package Imblearn untuk Oversampling atau Undersampling
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import Library
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

from imblearn.over_sampling import SMOTE, ADASYN

In [ ]:
# Import data
data = pd.read_excel('hasil_preprocessing_before_covid.xlsx')
data

,Unnamed: 0,Ulasan,Sentiment,Positive Probability,Negative Probability,clean,Case_Folding_&_Clean_Emoji,slang_word,Tokenizing,Filtering,Stemming
0,0,"Fasilitas hotel bagus, kamar mewah, protokol k...",Positive,0.300709,0.209502,Fasilitas hotel bagus kamar mewah protokol kov...,fasilitas hotel bagus kamar mewah protokol kov...,fasilitas hotel bagus kamar mewah protokol kov...,"['fasilitas', 'hotel', 'bagus', 'kamar', 'mewa...","['fasilitas', 'hotel', 'bagus', 'kamar', 'mewa...",fasilitas hotel bagus kamar mewah protokol kov...
1,1,"Makanan enak dan harga sangat terjangkau, mantap?",Positive,0.304822,0.171673,Makanan enak dan harga sangat terjangkau mantap,makanan enak dan harga sangat terjangkau mantap,makanan enak dan harga sangat terjangkau mantap,"['makanan', 'enak', 'dan', 'harga', 'sangat', ...","['makanan', 'enak', 'harga', 'terjangkau', 'ma...",makan enak harga jangkau mantap
2,2,"Pelayanan hotel sangat baik, kebersihan sangat...",Positive,0.273015,0.174779,Pelayanan hotel sangat baik kebersihan sangat ...,pelayanan hotel sangat baik kebersihan sangat ...,pelayanan hotel sangat baik kebersihan sangat ...,"['pelayanan', 'hotel', 'sangat', 'baik', 'kebe...","['pelayanan', 'hotel', 'kebersihan']",layan hotel bersih
3,3,Nyaman sekali menginap walaupun hanya satu har...,Positive,0.319150,0.160811,Nyaman sekali menginap walaupun hanya satu har...,nyaman sekali menginap walaupun hanya satu har...,nyaman sekali menginap walaupun hanya satu har...,"['nyaman', 'sekali', 'menginap', 'walaupun', '...","['nyaman', 'menginap', 'pelayanannya', 'memuas...",nyaman inap layan muas kesini
4,4,"Kamarnya luas nyaman dan bersih, pemandangan h...",Positive,0.302786,0.188203,Kamarnya luas nyaman dan bersih pemandangan ho...,kamarnya luas nyaman dan bersih pemandangan ho...,kamarnya luas nyaman dan bersih pemandangan ho...,"['kamarnya', 'luas', 'nyaman', 'dan', 'bersih'...","['kamarnya', 'luas', 'nyaman', 'bersih', 'pema...",kamar luas nyaman bersih pandang hotel bagus l...
...,...,...,...,...,...,...,...,...,...,...,...
2304,2317,Desain hotel sangat bagus tapi bau di dalam ho...,Positive,0.230125,0.215672,Desain hotel sangat bagus tapi bau di dalam ho...,desain hotel sangat bagus tapi bau di dalam ho...,desain hotel sangat bagus tapi bau di dalam ho...,"['desain', 'hotel', 'sangat', 'bagus', 'tapi',...","['desain', 'hotel', 'bagus', 'bau', 'hotel', '...",desain hotel bagus bau hotel lobi koridor kama...
2305,2318,Stafnya kurang profesional. 👎,Positive,0.307536,0.142843,Stafnya kurang profesional 👎,stafnya kurang profesional,stafnya kurang profesional,"['stafnya', 'kurang', 'profesional']","['stafnya', 'profesional']",staf profesional
2306,2319,Suasananya enak buat keluarga. Staf hotel oke ...,Positive,0.282238,0.200469,Suasananya enak buat keluarga Staf hotel oke s...,suasananya enak buat keluarga staf hotel oke s...,suasananya enak buat keluarga staf hotel oke s...,"['suasananya', 'enak', 'buat', 'keluarga', 'st...","['suasananya', 'enak', 'keluarga', 'staf', 'ho...",suasana enak keluarga staf hotel oke layan bre...
2307,2320,"Ruangan itu luas, staf sangat membantu, taman ...",Positive,0.277079,0.183748,Ruangan itu luas staf sangat membantu taman be...,ruangan itu luas staf sangat membantu taman be...,ruangan itu luas staf sangat membantu taman be...,"['ruangan', 'itu', 'luas', 'staf', 'sangat', '...","['ruangan', 'luas', 'staf', 'membantu', 'taman...",ruang luas staf bantu taman main tunggang kuda...


In [ ]:
# Melihat perbandingan Label
data.Sentiment.value_counts()

Positive    2101
Negative     208
Name: Sentiment, dtype: int64

TF-IDF (Term Frequency-Inverse Document Frequency) adalah sebuah metode yang digunakan dalam analisis teks untuk mengevaluasi pentingnya sebuah kata dalam suatu dokumen atau korpus berdasarkan frekuensi kemunculannya dan sebarannya di seluruh dokumen.

Dalam konteks analisis sentimen, TF-IDF digunakan untuk menghitung tingkat kepentingan kata-kata dalam teks yang berkaitan dengan sentimen yang ingin dianalisis (misalnya, positif atau negatif). Metode ini membantu dalam mengidentifikasi kata-kata kunci yang paling berperan dalam membentuk sentimen dalam teks.

Secara umum, TF-IDF terdiri dari dua komponen utama:

1. Term Frequency (TF): Merupakan pengukuran frekuensi kemunculan suatu kata dalam teks. Frekuensi ini dapat dihitung menggunakan metode sederhana seperti menghitung jumlah kata yang muncul dalam teks atau menggunakan metode yang lebih kompleks seperti skema pembobotan logaritmik untuk menekankan kata-kata yang muncul lebih sering.

2. Inverse Document Frequency (IDF): Merupakan pengukuran sejauh mana kata tersebut umum atau jarang muncul di seluruh korpus teks yang dianalisis. IDF digunakan untuk memberikan bobot yang lebih tinggi pada kata-kata yang jarang muncul, sehingga kata-kata tersebut dianggap lebih penting dalam analisis sentimen.

Dengan menggabungkan komponen TF dan IDF, nilai TF-IDF untuk suatu kata dapat dihitung dengan mengalikan frekuensi kemunculan kata tersebut (TF) dengan bobot invers dari frekuensi dokumen kata tersebut (IDF). Kata-kata dengan nilai TF-IDF yang tinggi dianggap lebih relevan dan berperan penting dalam analisis sentimen.

Penggunaan TF-IDF dalam analisis sentimen membantu dalam mengidentifikasi kata-kata kunci yang paling signifikan dalam membentuk sentimen dalam teks. Kata-kata dengan nilai TF-IDF tinggi sering kali memiliki pengaruh yang lebih besar terhadap penentuan sentimen suatu teks.

In [ ]:
# Pisahkan Data
X = data['Stemming']
y = data['Sentiment']

# Feature Engineering dengan TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Data Testing = 20% dan Data Training = 80%
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1847, 4096), (462, 4096), (1847,), (462,))

Oversampling adalah salah satu metode yang digunakan dalam pemrosesan data tidak seimbang (imbalanced data) untuk menangani ketidakseimbangan kelas. Ketidakseimbangan kelas terjadi ketika jumlah sampel dalam satu kelas jauh lebih sedikit dibandingkan dengan kelas lainnya.

SMOTE (Synthetic Minority Over-sampling Technique) adalah salah satu teknik oversampling yang umum digunakan. Metode ini bekerja dengan membuat sampel sintetis baru dari kelas minoritas dengan menggabungkan sampel-sampel yang sudah ada. SMOTE memilih beberapa sampel dari kelas minoritas dan menggabungkan mereka dengan tetangga-tetangga terdekat mereka. Dalam proses ini, SMOTE menghitung perbedaan antara sampel yang dipilih dan tetangga-tetangganya, dan kemudian menghasilkan sampel sintetis baru dengan menggabungkan fitur-fitur dari sampel yang dipilih dengan penyesuaian proporsional terhadap perbedaan tersebut.

Dengan menggunakan SMOTE, jumlah sampel dalam kelas minoritas dapat ditingkatkan secara sintetis sehingga mencapai tingkat kesetimbangan yang lebih baik dengan kelas mayoritas. Hal ini membantu mengurangi bias yang mungkin terjadi saat melatih model pada data yang tidak seimbang.

Penerapan SMOTE dalam pemrosesan data dapat dilakukan dengan menggunakan library atau paket khusus seperti imbalanced-learn di Python, yang menyediakan implementasi SMOTE dan metode-metode oversampling lainnya.

In [ ]:
# Membuat objek SMOTE dengan parameter yang diatur
smote = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
X_train_resampled.shape, y_train_resampled.shape

((3368, 4096), (3368,))

Code yang diberikan adalah implementasi SMOTE pada data training untuk melakukan oversampling. Berikut adalah penjelasan dari code dan hasilnya:

1. `smote = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)`: Membuat objek SMOTE dengan parameter yang telah diatur. Parameter `sampling_strategy='auto'` mengindikasikan bahwa kita ingin menyesuaikan jumlah sampel dari kelas minoritas secara otomatis. Parameter `k_neighbors=5` menentukan jumlah tetangga yang akan digunakan untuk menghasilkan sampel sintetis baru. Parameter `random_state=42` digunakan untuk menghasilkan hasil yang konsisten secara acak.

2. `X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)`: Melakukan proses oversampling menggunakan metode SMOTE. Data training yang tidak seimbang (X_train dan y_train) digunakan sebagai input. Metode `fit_resample()` akan menghasilkan data training baru yang sudah diresampling. Hasilnya disimpan dalam variabel `X_train_resampled` dan `y_train_resampled`.

3. `X_train_resampled.shape, y_train_resampled.shape`: Menampilkan bentuk (shape) dari data training yang sudah diresampling. Dalam contoh ini, hasilnya adalah ((3368, 4096), (3368,)), yang berarti setelah resampling, jumlah sampel dalam data training menjadi 3368 dan jumlah fitur (kolom) menjadi 4096. Juga, bentuk dari vektor label y_train_resampled adalah (3368,), menunjukkan bahwa jumlah sampelnya juga menjadi 3368.

Dengan melakukan oversampling menggunakan SMOTE, jumlah sampel dalam kelas minoritas (sentimen negatif) telah ditingkatkan sehingga lebih seimbang dengan kelas mayoritas (sentimen positif). Hal ini akan membantu meningkatkan performa model dalam mengenali kelas minoritas dan mengurangi bias yang mungkin terjadi.

In [ ]:
# Membuat Model Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [ ]:
# Prediksi Model
y_pred = model.predict(X_test)
y_pred

array(['Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Negative',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positi

In [ ]:
# Melihat Confusion Matriks
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[  8  37]
 [  1 416]]


In [ ]:
# Menghasilkan classification report
classification_rep = classification_report(y_test, y_pred)

# Menampilkan classification report
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

    Negative       0.89      0.18      0.30        45
    Positive       0.92      1.00      0.96       417

    accuracy                           0.92       462
   macro avg       0.90      0.59      0.63       462
weighted avg       0.92      0.92      0.89       462



In [ ]:
# Melihat hasil Evalusasi Dari Pos Label Positif
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Positive')
recall = recall_score(y_test, y_pred, pos_label='Positive')
f_score = f1_score(y_test, y_pred, pos_label='Positive')

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F-score: {:.2f}".format(f_score))


Accuracy: 0.92
Precision: 0.92
Recall: 1.00
F-score: 0.96


In [ ]:
# # Melihat hasil Evalusasi Dari Pos Label Negatif
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Negative')
recall = recall_score(y_test, y_pred, pos_label='Negative')
f_score = f1_score(y_test, y_pred, pos_label='Negative')

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F-score: {:.2f}".format(f_score))

Accuracy: 0.92
Precision: 0.89
Recall: 0.18
F-score: 0.30
